|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating token embeddings<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge: Compare embeddings with RSA<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
# install gensim
!pip install gensim

In [ ]:
# import the api for the glove matrices
import gensim.downloader as api

import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

# adjust matplotlib defaults to personal preferences
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
# from https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html
info = api.info()
for model_name, model_data in sorted(info['models'].items()):
  print('%s (%d records): %s' % (
        model_name,
        model_data.get('num_records', -1),
        model_data['description'][:40] + '...'))

In [ ]:
# load two glove variants
glove50  = api.load('glove-wiki-gigaword-50')
glove300 = api.load('glove-wiki-gigaword-300')

# Exercise 1: Embeddings matrices for selected words

In [ ]:
# list of words for RSA
words = [ 'space','spaceship','planet','moon','star','galaxy',
          'chair','table','couch','stool','floor',
          'apple','banana','pear','kiwi','orange','peach','watermelon','starfruit','date'
        ]

# embeddings matrix for these words
embedmat50  = np.array([glove50[w] for w in words])
embedmat300 = np.array([glove300[w] for w in words])

In [ ]:
# check matrices sizes
print(f'Size of  "50" matrix: {embedmat50.shape}')
print(f'Size of "300" matrix: {embedmat300.shape}')

# sanity-check that they're really different
plt.figure(figsize=(10,4))
plt.plot(range(glove50.vector_size),embedmat50[0,:],'s-',markerfacecolor=[.6,.6,.9],label='50 dimensions')
plt.plot(range(glove300.vector_size),embedmat300[0,:],'o-',markerfacecolor=[.9,.9,.6],label='300 dimensions')
plt.gca().set(xlim=[-5,glove300.vector_size+5],xlabel='Dimension',ylabel='Value',title=f'Embeddings for "{words[0]}"')

plt.legend(fontsize=10)
plt.show()

# Exercise 2: Calculate and visualize cosine similarity matrices

In [ ]:
# normalize each vector to its norm (unit length)
E_50_norm  = embedmat50  / np.linalg.norm(embedmat50, axis=1,keepdims=True)
E_300_norm = embedmat300 / np.linalg.norm(embedmat300,axis=1,keepdims=True)

# cosine similarity matrices
cs_matrix_50  = E_50_norm  @ E_50_norm.T
cs_matrix_300 = E_300_norm @ E_300_norm.T

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,5))

# even dims
h = axs[0].imshow(cs_matrix_50,vmin=.1,vmax=.6,cmap='plasma')
axs[0].set(xticks=range(0,len(words),2),xticklabels=words[::2],yticks=range(1,len(words),2),yticklabels=words[1::2],
           title='Cossim matrix for 50 dims')
axs[0].tick_params(axis='x',labelrotation=90)
fig.colorbar(h,ax=axs[0],pad=.02)

# odd dims
h = axs[1].imshow(cs_matrix_300,vmin=.1,vmax=.6,cmap='plasma')
axs[1].set(xticks=range(0,len(words),2),xticklabels=words[::2],yticks=range(1,len(words),2),yticklabels=words[1::2],
           title='Cossim matrix for 300 dims')
axs[1].tick_params(axis='x',labelrotation=90)
fig.colorbar(h,ax=axs[1],pad=.02)

plt.tight_layout()
plt.show()

# Exercise 3: Calculate a category selectivity index

In [ ]:
# ratio of within-category CS to between-category CS

# first create a group mask based on word order
group = np.array([ [1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,3,3,3,3] ])
groupmask = np.triu( group.T@group ,1)
print(groupmask)

In [ ]:
# then extract the average means within-category, vs across-category
specnum50 = ( cs_matrix_50[groupmask==1].mean() + cs_matrix_50[groupmask==4].mean() + cs_matrix_50[groupmask==9].mean() ) /3
specden50 = ( cs_matrix_50[groupmask==2].mean() + cs_matrix_50[groupmask==3].mean() + cs_matrix_50[groupmask==6].mean() ) /3
specIndex50 = specnum50/specden50

# repeat for 300D
specnum300 = ( cs_matrix_300[groupmask==1].mean() + cs_matrix_300[groupmask==4].mean() + cs_matrix_300[groupmask==9].mean() ) /3
specden300 = ( cs_matrix_300[groupmask==2].mean() + cs_matrix_300[groupmask==3].mean() + cs_matrix_300[groupmask==6].mean() ) /3
specIndex300 = specnum300/specden300

print(f'Selectivity index is {specIndex50:.2f} for 50d, and {specIndex300:.2f} for 300d.')

In [ ]:
# and visualize!
plt.figure(figsize=(10,4))


### 50D data
yWithin = cs_matrix_50[(groupmask==1) | (groupmask==4) | (groupmask==9)]
yBetwen = cs_matrix_50[(groupmask==2) | (groupmask==3) | (groupmask==6)]

# plot all the data
plt.plot(np.random.normal(loc=0,scale=.02,size=len(yWithin)),yWithin,
         'ko',markersize=10,markerfacecolor=[.3,.6,.4],alpha=.5)
plt.plot(np.random.normal(loc=1,scale=.02,size=len(yBetwen)),yBetwen,
         'ks',markersize=10,markerfacecolor=[.3,.6,.4],alpha=.5)

# and the means
plt.plot([-.2,.2],[yWithin.mean(),yWithin.mean()],'k-',linewidth=5,zorder=-3)
plt.plot([.8,1.2],[yBetwen.mean(),yBetwen.mean()],'k-',linewidth=5,zorder=-3)



### repeat for 300D data
yWithin = cs_matrix_300[(groupmask==1) | (groupmask==4) | (groupmask==9)]
yBetwen = cs_matrix_300[(groupmask==2) | (groupmask==3) | (groupmask==6)]

plt.plot(np.random.normal(loc=3,scale=.02,size=len(yWithin)),yWithin,
         'ro',markersize=10,markerfacecolor=[.3,.4,.6],alpha=.5)
plt.plot(np.random.normal(loc=4,scale=.02,size=len(yBetwen)),yBetwen,
         'rs',markersize=10,markerfacecolor=[.3,.4,.6],alpha=.5)
plt.plot([2.8,3.2],[yWithin.mean(),yWithin.mean()],'k-',linewidth=5,zorder=-3)
plt.plot([3.8,4.2],[yBetwen.mean(),yBetwen.mean()],'k-',linewidth=5,zorder=-3)


plt.gca().set(xlim=[-1,5],xticks=[0,1,3,4],xticklabels=['50-D\nwithin','50-D\nbetween','300-D\nwithin','300-D\nbetween'],
              ylabel='Cosine similarity',title='Cosine similarities within vs. across categories')
plt.show()

# Exercise 4: Quantitative comparison via RSA

In [ ]:
# extract the upper-triangular elements
unique_50  = cs_matrix_50[np.triu_indices_from(cs_matrix_50, k=1)]
unique_300 = cs_matrix_300[np.triu_indices_from(cs_matrix_300, k=1)]

# Pearson correlation
r = np.corrcoef(unique_50,unique_300)[0,1]

# plot
plt.plot(unique_50,unique_300,'ks',markerfacecolor=[.7,.4,.4,.7])
plt.gca().set(xlim=[-.6,1],ylim=[-.6,1],xlabel='50-D cosine similarities',ylabel='300-D cosine similarities',
              title=f'Correlation (RSA score): r = {r:.3f}')
plt.grid(linestyle='--',color=[.8,.8,.8])
plt.show()

# Exercise 5: Cluster with tSNE+DBscan

In [ ]:
def tSNE_dbscan(E):

  ### t-SNE
  tsne = TSNE(n_components=2,perplexity=5)
  tsne_result = tsne.fit_transform(E)

  ### dbscan
  clustmodel = DBSCAN(eps=20,min_samples=3).fit(tsne_result)
  groupidx = clustmodel.labels_

  # number of clusters
  nclust = max(groupidx)+1 # +1 for indexing

  return (tsne_result,(clustmodel,groupidx,nclust))

In [ ]:
### plotting
_,axs = plt.subplots(1,2,figsize=(10,4))

grouplabels = []

for datai,data in enumerate([E_50_norm,E_300_norm]):

  # call clustering function
  (tsne_result,(clustmodel,groupidx,nclust)) = tSNE_dbscan(data)
  grouplabels.append(groupidx)

  # compute cluster centers
  cents = np.zeros((nclust,2))
  for ci in range(nclust):
    cents[ci,0] = np.mean(tsne_result[groupidx==ci,0])
    cents[ci,1] = np.mean(tsne_result[groupidx==ci,1])

  # draw lines from each data point to the centroids of each cluster
  lineColors = 'rkbgm'
  for ii in range(len(tsne_result)):
    if groupidx[ii]==-1:
      axs[datai].plot(tsne_result[ii,0],tsne_result[ii,1],'k+')
    else:
      axs[datai].plot([ tsne_result[ii,0], cents[groupidx[ii],0] ],[ tsne_result[ii,1], cents[groupidx[ii],1] ],lineColors[groupidx[ii]])

  # now draw the raw data in different colors
  for i in range(nclust):
    axs[datai].plot(tsne_result[groupidx==i,0],tsne_result[groupidx==i,1],'o',markerfacecolor=lineColors[i])

  # and now plot the centroid locations
  axs[datai].plot(cents[:,0],cents[:,1],'kd',markerfacecolor=[.8,.7,.1],markersize=10)
  titlepart = ['50','300'][datai]
  axs[datai].set(xlabel='tSNE axis 1',ylabel='tSNE axis 2',title=f'tSNE+DBscan on {titlepart} dimensions')


plt.tight_layout()
plt.show()

In [ ]:
for i,labs in enumerate(grouplabels):

  titlepart = ['50','300'][i]
  print(f'\n** Clusters in {titlepart}-D:')

  # loop over cluster IDs
  for cidx in range(-1,np.max(labs)+1):

    # find all the tokens in this group
    tokensInGroup = np.where(labs==cidx)[0]

    # print them out
    if cidx==-1:
      firstpart = '   Ungrouped tokens :'
    else:
      firstpart = f'   Tokens in group {cidx}:'
    print(firstpart,[ ''.join(words[t]) for t in tokensInGroup ])